In [1]:
import numpy as np
from scipy.sparse import lil_matrix
import matplotlib.pyplot as plt
from math import factorial as fact
import itertools
import progressbar
import copy
%matplotlib inline

class Basis:
    
    def bose(self,m,n):
        '''Возвращает базис для Бозе-статистики. m - число узлов, n - число частиц'''
        R = fact(n + m - 1)/fact(n)/fact(m - 1)
        b = np.zeros((R,m))
        b[0,m-1] = n
        for i in range(R-1):
            j = m - 1
            while j > 0:
                if b[i,j] in range(2,n+1) :
                    b[i+1,:] = b[i,:]
                    b[i+1,j] = 0
                    b[i+1,j-1] = b[i+1,j-1] + 1
                    b[i+1,m-1] = b[i,j] - 1
                    break
                elif b[i,j] > 0:
                    b[i+1,:] = b[i,:]
                    b[i+1,j-1] = b[i+1,j-1] + 1
                    b[i+1,j] = b[i,j] - 1
                    break
                j -= 1
        return b
   
    def limit_basis(self,m,n,n_max):
        '''Возвращает базис для Бозе-статистики с ограничением числа частиц на узле. 
        m - число узлов, n - число частиц, n_max - максимальное число частиц на узле.'''
        # Размерность базиса
        R = fact(n + m - 1)/fact(n)/fact(m - 1)
        b = self.bose(m,n)
        f = np.zeros((R,m))
        j = 0
        # Откиддываем функции, в которых на узлах частиц больше n_max
        for i in range(b.shape[0]):
            if any(b[i] > n_max): 
                continue
            else:
                f[j] = b[i]
                j += 1
        return f[:j]

    def bose_unsave(self, m, n):
        '''Возвращает базис для Бозе статистики с несохраняющимся числом частиц.
        m - число узлов, n - максимальное число частиц на узле
        '''
        return np.array( map(list, itertools.product(range(n+1),repeat=m)) )
    
    def fermi(self, m, n_up, n_down):
        '''Возвращает базис для Ферми-статистики с учетом спина.'''
        R = (fact(m)/fact(n_up)/fact(m-n_up))*(fact(m)/fact(n_down)/fact(m-n_down)) 
        fs = np.zeros((R,2*m))
        part_1 = self.limit_basis(m,n_up,1)
        if n_up == n_down:
            part_2 = copy.copy(part_1)
        else:
            part_2 = self.limit_basis(m,n_down,1)
        size_1, size_2 = part_1.shape[0], part_2.shape[0]
        for i in range(size_1):
            for j in range(size_2):
                fs[i*size_2+j] = np.concatenate((part_1[i],part_2[j]), axis=0)
        return fs
    def full_basis(self,m_d, m_c, m_b, n_d_up, n_d_down, n_c_up, n_c_down, n_max):
        '''m_d - число узлов в кластере, m_c - число узлов в Ферми ванне, m_b - число узлов в Бозе ванне,
        n_d_up - число частиц со спином вверх в кластере, n_d_down - число частиц со спином вниз в кластере, 
        n_c_up - число частиц со спином вверх в Ферми ванне, n_c_down - число частиц со спином вниз в Ферми ванне,
        n_max - максимум частиц на узле в Бозе ванне
        '''
        mtx_1 = self.fermi(m_d+m_c, n_d_up+n_c_up,n_d_down+n_c_down)
        mtx_2 = self.bose_unsave(m_b,n_max)
        size_1, size_2 = mtx_1.shape[0], mtx_2.shape[0]
        fb = np.zeros((size_1*size_2,mtx_1.shape[1]+m_b))
        for i in range(size_1):
            for j in range(size_2):
                fb[i*size_2+j] = np.concatenate((mtx_1[i],mtx_2[j]), axis=0)
        return fb

In [2]:
class Hamiltonian(Basis):
    #def __init__(self,m_d, m_c, m_b, n_d_up, n_d_down, n_c_up, n_c_down, n_max, e_c, V_cd, U_d, e_b, gamma_bd, t_d, V_d):
        #self.m_d, self.m_c, self.m_b = m_d, m_c, m_b
        #self.n_d_up, self.n_d_down, self.n_c_up, self.n_c_down, self.n_max = n_d_up, n_d_down, n_c_up, n_c_down, n_max
    def __init__(self,e_c, V_cd, U_d, e_b, gamma_bd, t_d, V_d):
        self.e_c = e_c
        self.V_cd = V_cd
        self.U_d = U_d
        self.e_b = e_b
        self.gamma_bd = gamma_bd
        self.t_d = t_d
        self.V_d = V_d
    # Опертор рождения * уничтожения
    def up_down(self, number_to_up, number_to_down, func): 
        '''Функция возвращает коэффициент и новую функцию'''
        function = copy.copy(func)
        if function[number_to_down] == 0:
            return [0,function]
        elif function[number_to_up] == 1:
            return [0,function]
        else:
            coef_down = np.sqrt(function[number_to_down])
            function[number_to_down] = function[number_to_down] - 1
            coef_up = np.sqrt(function[number_to_up] + 1)
            function[number_to_up] = function[number_to_up] + 1
            return [round(coef_down*coef_up,5), function]

    # Знак 
    def sign(self, left_edge,right_edge,func):
        '''Функция определяет знак перескока'''
        left_edge,right_edge = min(left_edge,right_edge), max(left_edge,right_edge)
        if sum(func[left_edge+1:right_edge])%2:
            return 1
        else:
            return -1
    # Операторы рождения и уничтожения для бозе-ванны 
    def up(self,number_to_up,n_max,func):
        function = copy.copy(func)
        if function[number_to_up] == n_max:
            return [0,function]
        else:
            coef_up = np.sqrt(function[number_to_up] + 1)
            function[number_to_up] = function[number_to_up] + 1
            return [round(coef_up,5), function]

    def down(self,number_to_down, func):
        function = copy.copy(func)
        if function[number_to_down] == 0:
            return [0,function]
        else:
            coef_down = np.sqrt(function[number_to_down])
            function[number_to_down] = function[number_to_down] - 1
            return [round(coef_down,5), function]
            
    def matrix(self,m_d, m_c, m_b, n_d_up, n_d_down, n_c_up, n_c_down, n_max):   
        bar = progressbar.ProgressBar()
        basis = self.full_basis(m_d, m_c, m_b, n_d_up, n_d_down, n_c_up, n_c_down, n_max)
        R = basis.shape
        # Задаем соседей
        neigbors_d = {4:[5,6], 5:[4,7], 6:[4,7], 7:[5,6]}
        # создаем матрицу
        H = lil_matrix((R[0],R[0]))
        
        # Создаем словарь собственных функций
        basis = basis.astype(int)
    
        key = [''.join(map(str,basis[i])) for i in range(R[0])]
        value = range(len(key))
        indexes = dict(zip(key,value))

        
        for i in bar(range(R[0])):
            # Диагональные элементы
            H[i,i] += sum(self.e_c[:m_c]*basis[i,0:m_c] + self.e_c[m_c:]*basis[i,(m_d+m_c):(m_d+2*m_c)]) + self.U_d*sum(basis[i,m_c:(m_d+m_c)]*basis[i,(m_d+2*m_c):2*(m_d+m_c)]) + sum(self.e_b*basis[i,-m_b:])
            for j in neigbors_d.keys():
                for k in neigbors_d[j]:
                    H[i,i] += V_d*(basis[i,j]+basis[i,j+m_c])*(basis[i,k]+basis[i,k+m_c])
            # Остальное
            iks = 0 # индекс перескока с ферми-ванны на узел и наоборот
            iq = 0 # индекс для бозонов
            for j in range(m_c,m_c+m_d): # узлы кластера
                for p in range(m_c): # узлы ферми-ванны
                    # перескок с ванны на кластер
                    H[i,indexes[''.join(map(str,self.up_down(j,p,basis[i])[1]))]] += V_cd[iks]*self.sign(p,j,basis[i])*self.up_down(j,p,basis[i])[0]
                    H[i,indexes[''.join(map(str,self.up_down(m_c+m_d+j,p+m_c+m_d,basis[i])[1]))]] += V_cd[iks+2*(m_c+m_d)]*self.sign(m_c+m_d+p,j+m_c+m_d,basis[i])*self.up_down(m_c+m_d+j,p+m_c+m_d,basis[i])[0]
                    # перескок с кластера на ванну
                    H[i,indexes[''.join(map(str,self.up_down(p,j,basis[i])[1]))]] += V_cd[iks]*self.sign(p,j,basis[i])*self.up_down(p,j,basis[i])[0]
                    H[i,indexes[''.join(map(str,self.up_down(p+m_c+m_d,m_c+m_d+j,basis[i])[1]))]] += V_cd[iks+2*(m_c+m_d)]*self.sign(m_c+m_d+p,j+m_c+m_d,basis[i])*self.up_down(p+m_c+m_d,m_c+m_d+j,basis[i])[0]
                    iks += 1
                # рождение/уничтожение бозонов на фоне частиц на кластере
                number_particle = basis[i,j] + basis[i,j+m_c+m_d]
                for q in range(2*(m_c+m_d),2*(m_c+m_d)+m_b):
                    H[i,indexes[''.join(map(str,self.up(q,n_max,basis[i])[1]))]] += gamma_bd[iq]*self.up(q,n_max,basis[i])[0]*number_particle
                    H[i,indexes[''.join(map(str,self.down(q,basis[i])[1]))]] += gamma_bd[iq]*self.down(q,basis[i])[0]*number_particle
                    iq += 1
                for f in neigbors_d[j]:
                    H[i,indexes[''.join(map(str,self.up_down(j,f,basis[i])[1]))]] += t_d*self.sign(j,f,basis[i])*self.up_down(j,f,basis[i])[0]
                    H[i,indexes[''.join(map(str,self.up_down(j+m_c+m_d,f+m_c+m_d,basis[i])[1]))]] += t_d*self.sign(j+m_c+m_d,f+m_c+m_d,basis[i])*self.up_down(j+m_c+m_d,f+m_c+m_d,basis[i])[0]
        return H

In [4]:
e_c = np.array([1,2,3,4]*2)
V_cd = np.array([1]*32)
U_d = 2
e_b = np.array([4,3,2,1])
gamma_bd = np.array([1.5]*16)
t_d = 1
V_d = 0.5

In [6]:
f = Hamiltonian(np.array([1,2,3,4]*2),np.array([1]*32),2,np.array([4,3,2,1]),np.array([1.5]*16),1,0.5).matrix(4,4,4,2,2,2,2,3)

100% |########################################################################|


In [7]:
f.shape

(1254400, 1254400)

In [9]:
M = f.tocoo()
H = np.column_stack((M.row,M.col,M.data))
np.savetxt('text_H.csv',H,delimiter=',')

In [3]:
import pandas as pd
data = pd.read_csv('text_H.csv',delimiter=',',header=None)
row = data[0].tolist()
col = data[1].tolist()
val = data[2].tolist()

In [5]:
import scipy.sparse as ss
H = ss.coo_matrix((val, (row, col)), shape=(1254400, 1254400))

In [13]:
from scipy.sparse.linalg import eigs
[E,V] = eigs(H,k=10)

In [15]:
E

array([ 146.66069784+0.j,  137.05865194+0.j,  136.64448267+0.j,
        136.64116845+0.j,  136.00737210+0.j,  135.55984049+0.j,
        135.56345538+0.j,  134.64347626+0.j,  134.94971757+0.j,
        134.52362328+0.j])